<a href="https://colab.research.google.com/github/SuppaBach/2Wats_Prediction/blob/master/2Wats_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

**Problem 3: The Tale of Two Wat** 

Chulalongkorn University's AI Academy 2020

2nd round screening exam


Candidate name : 
Suppawat Boonrach

*   s.boonrach@outlook.com
*   github.com/suppabach



There are three parts in this program

1.   Preparation
2.   Training
3.   Prediction





# Preparation

Extract file

In [0]:
import os
import zipfile

local_zip = '/watprakeaw_vs_watpo.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

Prepare directory

In [0]:
base_dir = '/tmp/watprakeaw_vs_watpo'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training WatPo pictures
train_WatPo_dir = os.path.join(train_dir, 'WatPo')

# Directory with our training WatPrakeaw pictures
train_WatPrakeaw_dir = os.path.join(train_dir, 'WatPrakeaw')

# Directory with our validation WatPo pictures
validation_WatPo_dir = os.path.join(validation_dir, 'WatPo')

# Directory with our validation WatPrakaew pictures
validation_WatPrakeaw_dir = os.path.join(validation_dir, 'WatPrakeaw')

In [63]:
train_WatPo_fnames = os.listdir(train_WatPo_dir)
print(train_WatPo_fnames[:10])

train_WatPrakeaw_fnames = os.listdir(train_WatPrakeaw_dir)
train_WatPrakeaw_fnames.sort()
print(train_WatPrakeaw_fnames[:10])

['po1.png', 'po2.png', 'po3.png', 'po4.png']
['prakeaw1.png', 'prakeaw2.png', 'prakeaw3.png', 'prakeaw4.png']


In [64]:
print('total training WatPo images:', len(os.listdir(train_WatPo_dir)))
print('total training WatPrakeaw images:', len(os.listdir(train_WatPrakeaw_dir)))
print('total validation WatPo images:', len(os.listdir(validation_WatPo_dir)))
print('total validation WatPrakeaw images:', len(os.listdir(validation_WatPrakeaw_dir)))

total training WatPo images: 4
total training WatPrakeaw images: 4
total validation WatPo images: 4
total validation WatPrakeaw images: 4


# Training

In [66]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

base_dir = '/tmp/watprakeaw_vs_watpo'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255) # เราจะรีสเกลให้แคบลงเพื่อที่จะได้เรียนรู้เร็วขึ้น

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory( # ตรงนี้จะสร้างออปเจ็คไว้เทรน
        train_dir,
        target_size=(img_width, img_height),
        batch_size=4,
        class_mode='binary')

validation_generator = datagen.flow_from_directory( # ตรงนี้จะสร้างออปเจ็คไว้ validation
        validation_dir,
        target_size=(img_width, img_height),
        batch_size=4,
        class_mode='binary')

model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

nb_epoch = 4 #จำนวน epoch
nb_train_samples = 4 #จำนวน sample ที่นำมาเทรน
nb_validation_samples = 1 #จำนวน sample ที่นำมา validate

model.fit_generator(
        train_generator,
        samples_per_epoch=nb_train_samples,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        nb_val_samples=nb_validation_samples)

#สอนเสร็จก็เซฟ
open("basic_cnn_20_epochs.h5", "wb")
model.save_weights('basic_cnn_20_epochs.h5')

Found 8 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(512, 512,...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65:

Epoch 1/4
1/1 [==============================] - 3s 3s/step - loss: 0.6842 - accuracy: 0.7500 - val_loss: 14.7628 - val_accuracy: 0.5000
Epoch 2/4
1/1 [==============================] - 2s 2s/step - loss: 31.4512 - accuracy: 0.2500 - val_loss: 9.4660 - val_accuracy: 0.5000
Epoch 3/4
1/1 [==============================] - 2s 2s/step - loss: 5.9690 - accuracy: 0.7500 - val_loss: 2.7692 - val_accuracy: 0.2500
Epoch 4/4
1/1 [==============================] - 2s 2s/step - loss: 3.3424 - accuracy: 0.2500 - val_loss: 4.9493 - val_accuracy: 0.2500


# Classification

In [75]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
import random

img_width, img_height = 512, 512

model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.load_weights('basic_cnn_20_epochs.h5')

# Let's prepare a random input image of a cat or dog from the training set.
WatPo_img_files = [os.path.join(train_cats_dir, f) for f in train_WatPo_fnames]
WatPrakeaw_img_files = [os.path.join(train_dogs_dir, f) for f in train_WatPrakeaw_fnames]
img_path = random.choice(WatPo_img_files + WatPrakeaw_img_files)

predictg = img_to_array(load_img(img_path)
prediction = model.predict_classes(predictg)
if prediction[0][0] == 1: print("WatPo")
else : print("WatPrakeaw")
 


SyntaxError: ignored